In [1]:
import sympy as sp

Sympy gives you the ability to use python to do symbolic calculations, as numpy does for numerics. Fisrtly, we can use some of IPythons built in functions to get easily readable and pretty print outs.

In [2]:
from IPython.display import display, Latex

def print_expression(S, name=""):
    """Print out a single sympy expression"""
    string = "$$" 
    if name!="": string = string + name + "=" 
    string = string + sp.latex(S) + "$$"
    return display(Latex(string))

The basic type in sympy is the symbols. Out of these we can make expressions, and manipulate them.

In [3]:
a, b, c, x, y, z = sp.symbols("a, b, c, x, y, z")
f = 2 + x**2*y + sp.exp(x + y)
print_expression(f, name="f(x, y)")

<IPython.core.display.Latex object>

In [4]:
f.diff(x).diff(y)

2*x + exp(x + y)

In [5]:
sp.integrate(f, (x, 0, a))

a**3*y/3 + 2*a - exp(y) + exp(a + y)

In [6]:
g = sp.lambdify([x, y], f) # make a regular python funciton
print(g(1, 1))

10.38905609893065


We can manipulate arbitrary order symbolic tensors with the Array class 

In [7]:
A = sp.Array([[x, x*y], [sp.sin(x), x**y]])
print_expression(A)
print_expression(A.diff(x))
B = sp.Array([[x**2, y - x], [y, sp.exp(y/x)]])
print_expression(A.applyfunc(lambda x : x**2))


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

Matrix product, or a more general contraction of indecies, is done by first a tenosr product, then a contraction. For example, the matrix product $AB$ can be done by:

In [8]:
from sympy import tensorproduct as tp
from sympy import tensorcontraction as tc

print_expression(tc(tp(A, B), (1, 2)))

<IPython.core.display.Latex object>

If the array is 2D, there is more funcitonality, for example eigenvectors, in the Matrix calss. This array is not quite as rich as the numpy array. You can numpy arrays with sympy symbols, if anything is missing.

Indexed objects gives the possibility manipulate tensors with symbolic indecies, wich ranges from $0$ to $n - 1$

In [9]:
# Horrible hack to print indexed objects, courtesy StackOverflow: https://stackoverflow.com/questions/61470842/strange-printing-in-sympy-for-indexed-variables
from sympy import *
from sympy.printing.latex import LatexPrinter

class CustomLatexPrinter(LatexPrinter):
    def _print_Idx(self, expr):
        return expr.name

    @classmethod
    def printer(cls, expr, **kwargs):
        return cls(kwargs).doprint(expr)

init_printing(use_latex='mathjax', latex_printer=CustomLatexPrinter.printer)

In [10]:
i, j, m, n = sp.symbols("i, j, m, n", integer=True)
i = sp.Idx(i, n)    # A symbolic index
j = sp.Idx(j, n)
z = sp.IndexedBase("z")    # An object with indecies, like a tensor
z[i].diff(z[j])

δ   
 i,j

In [11]:
f = sp.sqrt(sp.Sum(z[i]**2, (i, 0, n-1)))
f

         _____________
        ╱ n - 1       
       ╱   ___        
      ╱    ╲          
     ╱      ╲       2 
    ╱       ╱   z[i]  
   ╱       ╱          
  ╱        ‾‾‾        
╲╱        i = 0       

sp.doit(j<n)

Now, we come to the $\gamma_\mu$ - matricies. 


In [64]:
# More printing hacks. This allows the gamma matrices to get latex names
# Based partly on the original sympy code https://github.com/sympy/

class CustomLatexPrinter(LatexPrinter):

    def _printer_tensor_indices2(self, name, indices, index_map={}):
        out_str = name # This is slighly changed from the source code
        last_valence = None
        prev_map = None
        for index in indices:
            new_valence = index.is_up
            if ((index in index_map) or prev_map) and \
                    last_valence == new_valence:
                out_str += ","
            if last_valence != new_valence:
                if last_valence is not None:
                    out_str += "}"
                if index.is_up:
                    out_str += "{}^{"
                else:
                    out_str += "{}_{"
            out_str += self._print(index.args[0])
            if index in index_map:
                out_str += "="
                out_str += self._print(index_map[index])
                prev_map = True
            else:
                prev_map = False
            last_valence = new_valence
        if last_valence is not None:
            out_str += "}"
        return out_str

    def _print_Tensor(self, expr):
        name = ""
        try:
            name = expr.args[0]._name
        except: 
            name = expr.args[0].args[0].name

        indices = expr.get_indices()
        return self._printer_tensor_indices2(name, indices)

        def _print_TensMul(self, expr):
            print("aaaaa")
        
    @classmethod
    def printer(cls, expr, **kwargs):
        return cls(kwargs).doprint(expr)


init_printing(use_latex='mathjax', latex_printer=CustomLatexPrinter.printer)

In [74]:
class CustomLatexPrinter(LatexPrinter):
    def _print_TensMul(self, expr):
        return expr.name

    @classmethod
    def printer(cls, expr, **kwargs):
        return cls(kwargs).doprint(expr)

init_printing(use_latex='mathjax', latex_printer=CustomLatexPrinter.printer)

In [25]:
from sympy.physics.hep.gamma_matrices import GammaMatrix as G, LorentzIndex, gamma_trace, simplify_gamma_expression, kahane_simplify
G._name = "\gamma"
g = LorentzIndex.metric
g._name = "g"
eps = LorentzIndex.epsilon
eps._name = "\\varepsilon"

from sympy.tensor.tensor import TensorIndex, tensor_indices, TensorIndexType

i0,i1,i2,i3,i4,i5 = tensor_indices('i_0:6', LorentzIndex)
G(i0)

           i₀
GammaMatrix  
             

In [22]:
g(i0, i1)

      i₀i₁
metric    
          

In [56]:
G5 = 1j/ sp.factorial(4) * eps(-i0, -i1, -i2, -i3) * G(i0) * G(i1) * G(i2) * G(i3) 
# G5

In [29]:
kahane_simplify(G(i0) * G(i1) * G(i2) * G(i3) *  G(-i0))

              i₃            i₂            i₁
-2⋅GammaMatrix  ⋅GammaMatrix  ⋅GammaMatrix  
                                            

In [75]:
print(type(G(i0) * G(i1) * G(i2) * G(i3) * G5))
print(G(i0) * G(i1) * G(i2) * G(i3) * G5)

<class 'sympy.tensor.tensor.TensMul'>


TypeError: Invalid comparison of non-real 0.0416666666666667*I

In [39]:
eps = LorentzIndex.epsilon
eps._name = "\\varepsilon"
eps(i0, i1, i2, i3)

   i₀i₁i₂i₃
Eps        
           

In [17]:
from sympy.tensor.tensor import TensorIndexType, TensorHead
i = TensorIndex('i', LorentzIndex)
j = TensorIndex('j', LorentzIndex, is_up=False)
Lorentz = TensorIndexType('Lorentz', dummy_name='L')
T = TensorHead("T", [Lorentz, Lorentz])
print(type(T(i, j)))
T(i, j)

<class 'sympy.tensor.tensor.Tensor'>


 i 
T  
  j